# Country-level exposure

In this notebook, we will perform a **exposure analysis** for all available CI data within a country. The assessment is based on combining hazard data (e.g., flood depths) with OpenStreetMap feature data.

We will follow the steps outlined below to conduct the assessment:

1. **Loading the necessary packages:**  
   We will import the Python libraries required for data handling, analysis, and visualization.

2. **Loading the data:**  
   The infrastructure data (e.g., roads) and hazard data (e.g., flood depths) will be loaded into the notebook.

3. **Preparing the data:**  
   The infrastructure and hazard data will be processed and data gaps can be filled, if required.

4. **Performing the exposure assessment:**  
   We will overlay the hazard data with the feature information.

5. **Visualizing the results:**  
   Finally, we will visualize the estimated exposure using graphs and maps.

## 1. Loading the Necessary Packages
To perform the assessment, we are going to make use of several [python packages](https://docs.python.org/3/tutorial/modules.html#packages).

In case you run this in Google Colab, you will have to install the packages below **(remove the hashtag in front of them)**.

In [1]:
#!pip install damagescanner==0.9b4
#!pip install contextily
#!pip install exactextract
#!pip install osm-flex

In this step, we will import all the required Python libraries for data manipulation, spatial analysis, and visualization.

In [2]:
import os,sys
import rasterio
import shapely
import traceback
import warnings
import xarray as xr
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns

from tqdm import tqdm

from pathlib import Path
import matplotlib.pyplot as plt
import rioxarray
import contextily as cx

from osm_flex import download
from exactextract import exact_extract
from damagescanner.core import DamageScanner
from damagescanner.base_values import DICT_CIS_VULNERABILITY_FLOOD

warnings.simplefilter(action='ignore', category=FutureWarning)

### Specify the country of interest

Before we continue, we should specify the country for which we want to assess the damage. We use the ISO3 code for the country to download the OpenStreetMap data.

In [3]:
country_full_name = 'Togo'
country_iso3 = 'TGO'

## 2. Loading the Data
In this step, we will prepare and load two key datasets:

1. **Infrastructure data:**  
   This dataset contains information on the location and type of infrastructure (e.g., roads). Each asset may have attributes such as type, length, and replacement cost.

2. **Hazard data:**  
   This dataset includes information on the hazard affecting the infrastructure (e.g., flood depth at various locations).

### Infrastructure Data

We will perform this example analysis for Jamaica. To start the analysis, we first download the OpenStreetMap data from GeoFabrik. 

In [4]:
infrastructure_path = download.get_country_geofabrik(country_iso3)

We will not load the data directly, we will let the code itself read the information. It is important, however, to specificy which infrastructure systems you want to include. We do so in the list below:

In [5]:
asset_types = [
        "roads",
        "main_roads",
        "rail",
        "air",
        "telecom",
        "water_supply",
        "waste_solid",
        "waste_water",
        "education",
        "healthcare",
        "power",
        "gas",
        "food",
        "oil",
        "wastewater",
    ]

### Hazard Data
For this example, we make use of the flood data provided by [CDRI](https://giri.unepgrid.ch/map?list=explore).

We use a 1/100 flood map to showcase the approach.

In [6]:
hazard_map = xr.open_dataset("https://hazards-data.unepgrid.ch/global_pc_h100glob.tif", engine="rasterio")

### Ancilliary data for processing

In [7]:
world = gpd.read_file("https://github.com/nvkelso/natural-earth-vector/raw/master/10m_cultural/ne_10m_admin_0_countries.shp")
world_plot = world.to_crs(3857)

## 3. Preparing the Data

Clip the hazard data to the country of interest.

In [8]:
country_bounds = world.loc[world.ADM0_ISO == country_iso3].bounds
country_geom = world.loc[world.ADM0_ISO == country_iso3].geometry

In [9]:
hazard_country = hazard_map.rio.clip_box(minx=country_bounds.minx.values[0],
                     miny=country_bounds.miny.values[0],
                     maxx=country_bounds.maxx.values[0],
                     maxy=country_bounds.maxy.values[0]
                    )

## 4. Performing the Exposure Assessment
We will use the DamageScanner approach. This is a fully optimised damage calculation method, that can capture a wide range of inputs to perform a damage assessment.

In [ ]:
save_asset_results = {}
for asset_type in asset_types:
        save_asset_results[asset_type] = DamageScanner(hazard_country, 
                                                       infrastructure_path, 
                                                       curves=pd.DataFrame(), 
                                                       maxdam=pd.DataFrame()).exposure(
            asset_type=asset_type
        )

Overlay raster with vector: 100%|██████████████████████████████████████████████████████| 44/44 [00:00<00:00, 92.37it/s]
convert coverage to meters: 1it [00:00, 784.72it/s]
convert coverage to meters: 100%|████████████████████████████████████████████████████| 20/20 [00:00<00:00, 9728.18it/s]


In [ ]:
number_of_assets_exposed = {}
for asset_type in asset_types:
    number_of_assets_exposed[asset_type] = len(save_asset_results[asset_type])

In [ ]:
# Update data keys for improved readability
formatted_data = {key.replace('_', ' ').capitalize(): value for key, value in number_of_assets_exposed.items()}

# Create figure with updated title and formatted categories
fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.bar(formatted_data.keys(), formatted_data.values(), color='skyblue', edgecolor='black')

# Add labels and enhance aesthetics
ax.set_title(f"Infrastructure Exposure per System for {country_full_name}", fontsize=16, weight='bold')
ax.set_ylabel("Count", fontsize=14)
ax.set_xlabel("Categories", fontsize=14)
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.yticks(fontsize=12)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.grid(axis='y', linestyle='--', alpha=0.6)

# Show figure
plt.tight_layout()
plt.show()